In [1]:
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, UpSampling2D, concatenate


In [2]:
def conv_block(x_in, filters, batch_norm, kernel_size=(3,3)):
    x = Conv2D(filters, kernel_size, padding='same')(x_in)
    if batch_norm=='TRUE':
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size, padding='same')(x)
    if batch_norm=='TRUE':
        x = BatchNormalization()(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def conv_2d(x_in, filters, batch_norm, kernel_size=(3,3),acti ='relu'):
    x = Conv2D(filters, kernel_size, padding='same')(x_in)
    if batch_norm=='TRUE':
        x=BatchNormalization()(x)
    x= Activation(acti)(x)
    return x

def pool(x_in, pool_size=(2, 2), type='Max'):
    if type=='Max':
        p = MaxPooling2D(pool_size)(x_in)
    return p

def up(x_in, filters, merge, batch_norm, size=(2,2)):
    u = UpSampling2D(size)(x_in)
    conv = conv_block(u, filters, batch_norm)
    merge=concatenate([merge, conv],axis=-1)
    return merge

def Unet_01(input_layer):
    conv1 = conv_block(input_layer, filters=16, batch_norm='TRUE')
    pool1 = pool(conv1)

    conv2 = conv_block(pool1, filters=32, batch_norm='TRUE')
    pool2 = pool(conv2)

    conv3 = conv_block(pool2, filters=32, batch_norm='TRUE')
    pool3 = pool(conv3)

    conv4 = conv_block(pool3, filters=64, batch_norm='TRUE')
    pool4 = pool(conv4)

    conv5 = conv_2d(pool4, filters=128, batch_norm='TRUE')

    up1 = up(conv5,filters=128, merge=conv4, batch_norm='TRUE')
    conv6 = conv_2d(up1, filters=128, batch_norm='TRUE')

    up2 = up(conv6, filters=128, merge=conv3, batch_norm='TRUE')
    conv7 = conv_2d(up2, filters=128, batch_norm='TRUE')

    up3 = up(conv7, filters=64, merge=conv2, batch_norm='TRUE')
    conv8 = conv_2d(up3, filters=64, batch_norm='TRUE')

    up4 = up(conv8, filters=32, merge=conv1, batch_norm='TRUE')
    conv9 = conv_2d(up4, filters=32, batch_norm='TRUE')

    conv10 = conv_2d(conv9, filters=1, batch_norm='FALSE', acti='sigmoid')

    output_layer = conv10
    model = Model(input_layer, output_layer)

    return model

In [ ]:
model.summary()

In [22]:
# Define input layer
input_layer = Input(shape=(128,128,1))

In [23]:
# Create the model
model = Unet_01(input_layer)


In [13]:
# Load the weights from the .h5 file
model.load_weights(r'C:\Users\Shantanu\Desktop\projects\seg.weights.h5')

In [26]:
import numpy as np
import nibabel as nib

# Load the test image
test_img_path = 'tmp_img/test_img/1.nii.gz'
test_img = nib.load(test_img_path).get_fdata()

# Ensure the test image has the correct shape and type
test_img = np.expand_dims(test_img, axis=-1)  # Add channel dimension
test_img = np.expand_dims(test_img, axis=0)   # Add batch dimension

# Pad or crop along the last dimension to match the desired depth (60)
depth = 60
current_depth = test_img.shape[-2]
if current_depth < depth:
    pad_width = ((0, 0), (0, 0), (0, 0), (0, depth - current_depth), (0, 0))
    test_img = np.pad(test_img, pad_width, mode='constant', constant_values=0)
elif current_depth > depth:
    test_img = test_img[:, :, :, :depth, :]

# Make predictions using model.predict()
predicted_mask = model.predict(test_img)


ValueError: Exception encountered when calling Functional.call().

[1mInput 0 of layer "conv2d_26" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (1, 128, 128, 60)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(1, 128, 128, 60, 1), dtype=float32)
  • training=False
  • mask=None